In [1]:
import numpy as np
import pandas as pd
import sys
import os 
if os.path.basename(os.getcwd()) == 'notebooks':
    os.chdir('..')
import glob

import sys
print(sys.path)

from sglm import utils, glm_fit

#d1 - T430
#d2 - T434

['c:\\Users\\Livia\\Desktop\\photometry_liv\\photometry_preprocessing\\sabatini-glm-workflow\\notebooks', 'c:\\Users\\Livia\\.conda\\envs\\glm\\python311.zip', 'c:\\Users\\Livia\\.conda\\envs\\glm\\DLLs', 'c:\\Users\\Livia\\.conda\\envs\\glm\\Lib', 'c:\\Users\\Livia\\.conda\\envs\\glm', '', 'c:\\Users\\Livia\\.conda\\envs\\glm\\Lib\\site-packages', 'c:\\Users\\Livia\\.conda\\envs\\glm\\Lib\\site-packages\\win32', 'c:\\Users\\Livia\\.conda\\envs\\glm\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\Livia\\.conda\\envs\\glm\\Lib\\site-packages\\Pythonwin']


## Create a project

#### First, let's create a new project. The project directory will create a data and results folder and a config file.

#### You will need to edit the config file with the particular glm params you wish to use. Fields that are necessary to edit are: predictors, predictors_shift_bounds, response, and the glm_keyword_args.

#### You will also need to move your data into the data folder.

In [2]:
project_name = 'A2a_sh_noITI_glm'
project_dir = r'\\research.files.med.harvard.edu\\Neurobio\\MICROSCOPE\\Livia\\glm_output' # windows
# project_dir = r'/Volumes/Neurobio/MICROSCOPE/Livia/glm_output' # mac

utils.create_new_project(project_name, project_dir)

Project directory already exists!


'\\\\research.files.med.harvard.edu\\\\Neurobio\\\\MICROSCOPE\\\\Livia\\\\glm_output\\A2a_sh_noITI_glm\\config.yaml'

# Import and Format Data

Input data should conform to the following convention and be saved as a *.csv:

Indices / Unique Row Identifiers:
* SessionName -- Any order is acceptable
* TrialNumber-- Must be in chronological order, but does not need to start from zero
* Timestamp -- Must be in chronological order, but does not need to start from zero

Columns (Predictors + Responses):
* Predictors - binary
* Reponses - e.g. neural responses (analog or binary)

Example, shown below is dummy data depicting a trial_0 that last four response timestamps:
| SessionName | TrialNumber | Timestamp | predictor_1 | predictor_2 | predictor_3 | response_1 | response_2 |
| --- | --- | --- | --- | --- | --- | --- | --- |
| session_0 | trial_0 | -1 | 0 | 0 | 0 | 1 | 0.3 |
| session_0 | trial_0 | 0 | 0 | 0 | 0 | 0 | 1.4 |
| session_0 | trial_0 | 1 | 0 | 0 | 0 | 1 | 2.3 |
| session_0 | trial_0 | 2 | 0 | 1 | 0 | 1 | 0.3 |
| session_0 | trial_1 | -2 | 0 | 0 | 0 | 0 | 1.4 |
| session_0 | trial_1 | -1 | 0 | 0 | 0 | 1 | 2.3 |
| session_0 | trial_1 | 0 | 1 | 0 | 0 | 0 | 1.4 |
| session_0 | trial_1 | 1 | 0 | 0 | 0 | 1 | 2.3 |
| session_1 | trial_0 | 5 | 0 | 0 | 0 | 0 | 1.4 |
| session_1 | trial_0 | 6 | 1 | 0 | 0 | 1 | 2.3 |
| session_1 | trial_0 | 7 | 0 | 0 | 0 | 0 | 1.4 |
| session_1 | trial_0 | 8 | 0 | 0 | 0 | 1 | 2.3 |
| session_1 | trial_1 | 9 | 0 | 0 | 0 | 0 | 1.4 |
| session_1 | trial_1 | 10 | 0 | 0 | 0 | 1 | 2.3 |
....

#### Now, let's get set up to start our project

In [3]:
project_path = os.path.join(project_dir, project_name)
files = os.listdir(project_path)

assert 'data' in files, 'data folder not found! {}'.format(files)
assert 'results' in files, 'results folder not found! {}'.format(files)
assert 'config.yaml' in files, 'config.yaml not found! {}'.format(files)

#### If needed, use the following function to combine multiple sessions into one csv. You will need a filename you wish to call your output_csv

In [4]:
left = False
if left:
    # output_csv = 'output.csv'
    p_split = project_name.split("_")
    p_name = p_split[0][:len(p_split[0])-1]
    name = p_name +"_"+ p_split[1]
    print(name)
    output_csv = name +'Format.csv'
else: output_csv = project_name +'Format.csv'

output_csv
# utils.combine_csvs(project_path, output_csv)

'A2a_sh_noITI_glmFormat.csv'

#### Next, we'll open the data and set the columns you wish to use as fixed indices

In [5]:
input_file = os.path.join(project_path, 'data', output_csv)
index_col = ['SessionName', 'TrialNumber', 'Timestamp']

df = utils.read_data(input_file, index_col)

print('Your dataframe has {} rows and {} columns'.format(df.shape[0], df.shape[1]))
df

Your dataframe has 1408040 rows and 15 columns


Cue  Select_L  Select_R  ENLP_L  \
SessionName     TrialNumber Timestamp                                      
T434_2023_08_11 2824.0      351.519867   0.0         0         0       0   
                            351.540511   0.0         0         0       0   
                            351.561155   0.0         0         0       0   
                            351.581798   0.0         0         0       0   
                            351.602442   0.0         0         0       0   
...                                      ...       ...       ...     ...   
T433_2023_08_01 658.0       1594.485473  0.0         0         0       0   
                            1594.506117  0.0         0         0       0   
                            1594.526761  0.0         0         0       0   
                            1594.547405  0.0         0         0       0   
                            1594.568049  0.0         0         0       0   

                                         ENLP_R  Consumption_R_R  \
SessionName     TrialNumber Timestamp                              
T434_2023_08_11 2824.0      351.519867        0              0.0   
                            351.540511        0              0.0   
                            351.561155        0              0.0   
                            351.581798        0              0.0   
                            351.602442        0              0.0   
...                                         ...              ...   
T433_2023_08_01 658.0       1594.485473       0              0.0   
                            1594.506117       0              0.0   
                            1594.526761       0              0.0   
                            1594.547405       0              0.0   
                            1594.568049       0              0.0   

                                         Consumption_R_L  Cons_more_R_R  \
SessionName     TrialNumber Timestamp                                     
T434_2023_08_11 2824.0      351.519867               0.0            0.0   
                            351.540511               0.0            0.0   
                            351.561155               0.0            0.0   
                            351.581798               0.0            0.0   
                            351.602442               0.0            0.0   
...                                                  ...            ...   
T433_2023_08_01 658.0       1594.485473              0.0            0.0   
                            1594.506117              0.0            0.0   
                            1594.526761              0.0            0.0   
                            1594.547405              0.0            0.0   
                            1594.568049              0.0            0.0   

                                         Cons_more_R_L  Consumption_UR_R  \
SessionName     TrialNumber Timestamp                                      
T434_2023_08_11 2824.0      351.519867             0.0               0.0   
                            351.540511             0.0               0.0   
                            351.561155             0.0               0.0   
                            351.581798             0.0               0.0   
                            351.602442             0.0               0.0   
...                                                ...               ...   
T433_2023_08_01 658.0       1594.485473            0.0               0.0   
                            1594.506117            0.0               0.0   
                            1594.526761            0.0               0.0   
                            1594.547405            0.0               0.0   
                            1594.568049            0.0               0.0   

                                         Consumption_UR_L  Cons_more_UR_R  \
SessionName     TrialNumber Timestamp                                       
T434_2023_08_11 2824.0      351.519867          

#### You can now explore and add to the dataframe. As an example, you may want to add various "predictors" or "features" to explore. You can use the example below as inspiration

In [6]:
# #Identify the individual licks that have specific meaning in the tasks: 
# #lick 1, lick 2 and lick 3 are "operant licks" on different training days
# #licknon1-3 are all the other licks

# df_source = df.copy()
# srs_lick = df_source.groupby(['SessionName', 'TrialNumber'])['Lick'].cumsum()
# srs_lick_count = srs_lick * df_source['Lick']
# df_lick_count_dummies = pd.get_dummies(srs_lick_count).drop(0, axis=1)
# df_lick_count_dummies = df_lick_count_dummies[[1,2,3]]
# df_lick_count_dummies['non1-3'] = df_source['Lick'] - df_lick_count_dummies.sum(axis=1)
# df_lick_count_dummies.columns = [f'lick_{original_column_name}' for original_column_name in df_lick_count_dummies.columns]

# # Columns lick and lick_1, lick_2, lick_3, lick_non-13 should not all be used together
# # as predictors because of multicollinearity.
# df_source = pd.concat([df_source, df_lick_count_dummies], axis=1)
# df_source

# assert np.all(df_source['Lick'] == df_source[['lick_1', 'lick_2', 'lick_3', 'lick_non1-3']].sum(axis=1)), 'Column lick should equal the sum of all other lick columns.'

#### Friendly reminder, the df we have imported is mutli-index, meaning, it's organization is dependent on 3-columns that we have set in index_col. Therefore, we can use "groupby" if you need to split the organization. 

In [7]:
# reIndex = df_source.groupby(level=[0, 1])

## Load your fitting paramaters and set up your train/test data

In [8]:
config_file = os.path.join(project_path, 'config.yaml')
config = utils.load_config(config_file)
config

{'Project': {'project_name': 'A2a_shuffled_glm',
  'project_path': '/Volumes/Neurobio/MICROSCOPE/Livia/glm_output/A2a_shuffled_glm'},
 'glm_params': {'glm_keyword_args': {'alpha': 0.0025,
   'cv': 5,
   'fit_intercept': True,
   'l1_ratio': 0.00025,
   'max_iter': 1000,
   'n_alphas': 100,
   'n_jobs': -1,
   'score_metric': 'r2',
   'selection': 'cyclic',
   'warm_start': False},
  'predictors': ['Cue',
   'Select_L',
   'Select_R',
   'ENLP_L',
   'ENLP_R',
   'Consumption_R_R',
   'Cons_more_R_R',
   'Consumption_R_L',
   'Cons_more_R_L',
   'Consumption_UR_R',
   'Cons_more_UR_R',
   'Consumption_UR_L',
   'Cons_more_UR_L'],
  'predictors_shift_bounds': {'Cue': [-75, 75],
   'Select_L': [-75, 75],
   'Select_R': [-75, 75],
   'ENLP_L': [-75, 75],
   'ENLP_R': [-75, 75],
   'Consumption_R_R': [-75, 75],
   'Cons_more_R_R': [-75, 75],
   'Consumption_R_L': [-75, 75],
   'Cons_more_R_L': [-75, 75],
   'Consumption_UR_R': [-75, 75],
   'Cons_more_UR_R': [-75, 75],
   'Consumption_UR_L'

#### Shift responses and predictors. If you do not want to shift your predictors by an amount you set, feel free to comment out the entire "predictors_shift_bounds" in config.yaml. We will then use the default set when we created the config file.

In [9]:
response_shift, df_predictors_shift, shifted_params = glm_fit.shift_predictors(config, df)
print('Your dataframe was shifted using: {}'.format(shifted_params))
# sparse array -> shift


Your dataframe was shifted using: [('Cue', [-75, 75]), ('Select_L', [-75, 75]), ('Select_R', [-75, 75]), ('ENLP_L', [-75, 75]), ('ENLP_R', [-75, 75]), ('Consumption_R_R', [-75, 75]), ('Cons_more_R_R', [-75, 75]), ('Consumption_R_L', [-75, 75]), ('Cons_more_R_L', [-75, 75]), ('Consumption_UR_R', [-75, 75]), ('Cons_more_UR_R', [-75, 75]), ('Consumption_UR_L', [-75, 75]), ('Cons_more_UR_L', [-75, 75])]


In [10]:
#create sparse array - by row
import scipy
sparse_df = scipy.sparse.csr_array(df_predictors_shift)

In [11]:
sparse_df

<1350438x1963 sparse array of type '<class 'numpy.float64'>'
	with 14350819 stored elements in Compressed Sparse Row format>

In [12]:
response_shift 
temp = response_shift.values.flatten()

### Create your test/train datasets

In [13]:
# X_train,X_test, y_train, y_test = glm_fit.split_data(df_predictors_shift, temp, config) #changed from df_predictors_shift to sparse_df! 
X_train,X_test, y_train, y_test = glm_fit.split_data(sparse_df, temp, config) #changed from df_predictors_shift to sparse_df! 

print('Training data has {} rows and {} columns'.format(X_train.shape[0], X_train.shape[1]))
print('Testing data has {} rows and {} columns'.format(X_test.shape[0], X_test.shape[1]))


Training data has 1080350 rows and 1963 columns
Testing data has 270088 rows and 1963 columns


In [14]:
# y_test1 = (y_test.values).flatten()


## Now, we're ready to run our GLM!

### We have two different options. If you know which params you would like to use, you can use the glm_fit.fit_glm function. If you would like tune your hyperparams to determine which are the best to use, you can use the glm_fit.fit_tuned_glm function. 

In [15]:
# # Fit the model
model, y_pred, score, beta, intercept, sparse_beta = glm_fit.fit_glm(config, X_train, X_test, y_train, y_test)
print('Your model can account for {} percent of your data'.format(score*100))
modeltype = "fit"

ypred: [-0.212907    0.75664413  0.54706519 ...  0.65708319 -0.29851659
 -0.44439106]
y: [-0.59453422 -0.94069645  0.24037615 ...  0.51996924 -0.9316393
 -0.95964176]
Your model can account for 18.33482477078866 percent of your data


In [16]:
# Fit the model with cross validation: remember, your alphas and l1_ratios should be lists
# tuned_model, y_pred, score, beta, best_params = glm_fit.fit_tuned_glm(config, X_train, X_test, y_train, y_test)
# print('Your model can account for {} percent of your data'.format(score*100))
# modeltype = 'tuned'

In [ ]:
# Fit the Ridge model
# ridge_model, y_pred, score, beta, intercept = glm_fit.fit_ridge(config, X_train, X_test, y_train, y_test)
# print('Your model can account for {} percent of your data'.format(score*100))


In [ ]:
# Fit the Ridge model with cross validation: remember, your alphas should be a list
# tuned_ridge_model, y_pred, score, beta, best_params = glm_fit.fit_tuned_ridge(config, X_train, X_test, y_train, y_test)
# print('Your model can account for {} percent of your data'.format(score*100))

In [17]:
%matplotlib inline
import matplotlib.pyplot as plt

In [18]:
def plot_spikes_with_prediction(spikes, predicted_spikes, dt,
                                nt=50, t0=120, **kws):
  """Plot actual and predicted spike counts.

  Args:
    spikes (1D array): Vector of actual spike counts
    predicted_spikes (1D array): Vector of predicted spike counts
    dt (number): Duration of each time bin.
    nt (number): Number of time bins to plot
    t0 (number): Index of first time bin to plot.
    show (boolean): To plt.show or not the plot.
    kws: Pass additional keyword arguments to plot()

  """
  t = np.arange(t0, t0 + nt) * dt

  f, ax = plt.subplots()
  lines = ax.stem(t, spikes[:nt])
  plt.setp(lines, color=".5")
  lines[-1].set_zorder(1)
  kws.setdefault("linewidth", 3)
  yhat, = ax.plot(t, predicted_spikes[:nt], **kws)
  ax.set(
      xlabel="Time (s)",
      ylabel="Spikes",
  )
  ax.yaxis.set_major_locator(plt.MaxNLocator(integer=True))
  ax.legend([lines[0], yhat], ["Spikes", "Predicted"])
  plt.show()

# plot_spikes_with_prediction(y_test, y_pred, df.Timestamp[1] - df.Timestamp[0])
# fig = plt.figure(1)
# scale = 200
# plt.plot(np.arange(0, len(y_test))[0:scale], y_test[0:scale], color='lightblue', label='test')
# plt.plot(np.arange(0, len(y_test))[0:scale], y_pred[0:scale], color='k', label='y_pred')


In [19]:
print(y_test.shape, y_pred.shape)


(270088,) (270088,)


In [20]:
# best_params


NameError: name 'best_params' is not defined

acc:

## Save your outputs

In [21]:
#Create your model dictonary, this should include all the information you wish to save
if modeltype == "fit":
    model_dict = {'model': model,
              'y_pred': y_pred,
              'score': score,
              'beta': beta,
              'intercept': intercept,
              'sparse_beta': sparse_beta,}
if modeltype == 'tuned':
    model_dict = {'model': tuned_model,
              'y_pred': y_pred,
              'score': score,
              'beta': beta,
              'best_params': best_params,}
    

#Save your model dictionary
model_path = project_path + '/models'
model_name = project_name + '_model.pkl'
model_full_path = os.path.join(model_path, model_name)
import pickle
with open(model_full_path, 'wb') as f:
    pickle.dump(model_dict, f)
        
# glm_fit.save_model(model_dict, project_path)

## Generate and save figures

In [22]:
save_img = os.path.join(project_path)

glm_fit.plot_and_save(save_img, config, y_pred, y_test, beta, df_predictors_shift, format(score*100))

saved to: \\research.files.med.harvard.edu\\Neurobio\\MICROSCOPE\\Livia\\glm_output\A2a_sh_noITI_glm/results/model_fit.png
